# Cross-validation of dense and SparsePauliOp implementations

In [1]:
import sys
sys.path.append('/home/iiyama/src/qt4hep')
import numpy as np
import scipy
import matplotlib.pyplot as plt
from qiskit.quantum_info import SparsePauliOp

from qt4hep.staggered_fermion import *
from qt4hep.spo import *
from qt4hep.dense import *

In [2]:
num_sites = 8
half_lat = num_sites // 2
mass = 1.
lsp = 0.5

In [3]:
rapidity, wavenumber = get_rapidity(num_sites, lsp, mass, with_wn=True)

## $\Phi$ operators

In [4]:
phi_spo = jw_annihilator_spo(num_sites)
phi_dense = jw_annihilator_dense(num_sites)

np.allclose(np.array([op.to_matrix() for op in phi_spo]), phi_dense)

True

## Free-fermion Hamiltonian

In [5]:
hopping_spo = staggered_hopping_term_spo(num_sites, lsp)
hopping_dense = staggered_hopping_term_dense(num_sites, lsp)
np.allclose(hopping_spo.to_matrix(), hopping_dense)

True

In [6]:
mass_spo = staggered_mass_term_spo(num_sites, mass)
mass_dense = staggered_mass_term_dense(num_sites, mass)
np.allclose(mass_spo.to_matrix(), mass_dense)

True

## Fock-space operators

In [7]:
fock_spo = phi_to_ab_spo(phi_spo, rapidity, wavenumber)
fock_dense = phi_to_ab_dense(phi_dense, rapidity, wavenumber)
np.allclose(np.array([op.to_matrix() for op in fock_spo]), fock_dense)

True

## Inverse transformation from $a, b^{\dagger}$ to $\Phi$

In [ ]:
phi_test = ab_to_phi_dense(fock_dense, rapidity, wavenumber)
np.allclose(phi_dense, phi_test)

True

In [9]:
phi_test = ab_to_phi_spo(fock_spo, rapidity, wavenumber)
np.allclose(np.array([op.to_matrix() for op in phi_spo]), np.array([op.to_matrix() for op in phi_test]))

True

## Free Hamiltonian diagonalization

In [10]:
free_h_spo = schwinger_hamiltonian_spo(num_sites, lsp, mass, 0.).to_matrix()
fockdag_dense = dagger(fock_dense)
adaga = np.einsum('nij,njk->nik', fockdag_dense[:half_lat], fock_dense[:half_lat])
bbdag = np.einsum('nij,njk->nik', fock_dense[half_lat:], fockdag_dense[half_lat:])
energy = mass * np.cosh(rapidity)
free_h_diag = np.sum(energy[:, None, None] * (adaga - bbdag), axis=0)

np.allclose(free_h_diag, free_h_spo)

True

## Simultaneous diagonalizability of the Fock number ops

In [11]:
fock_label = np.array([1] + [0] * (2 * half_lat - 2) + [1])
numop = np.einsum('nij,njk->nik', fockdag_dense, fock_dense)

eye = np.eye(2 ** num_sites, dtype=np.complex128)
proj = eye
for occ, op in zip(fock_label, numop):
    if occ == 0:
        op = eye - op
    proj = op @ proj

eigvals, eigvecs = np.linalg.eigh(proj)
indices = np.nonzero(np.isclose(eigvals, 1))[0]
indices.shape[0] == 1

True

In [12]:
state = cleaned(eigvecs[:, indices[0]])
state /= np.sqrt(np.sum(np.square(np.abs(state))))
indices = np.nonzero(state)[0]
terms = []
for idx in indices:
    binary = (np.array(idx) >> np.arange(num_sites)[::-1]) % 2
    coeff = state[idx].real if np.isclose(state[idx].imag, 0.) else state[idx]
    terms.append((coeff, ''.join(f'{b}' for b in binary)))
print(' + '.join(f'{term[0]:.3f}|{term[1]})' for term in terms))

0.081+0.004j|00001111) + -0.085+0.077j|00011011) + 0.004-0.081j|00011110) + 0.010-0.222j|00101011) + 0.081+0.004j|00101101) + -0.010+0.222j|00101110) + -0.085+0.077j|00111001) + 0.010-0.222j|00111010) + -0.004+0.081j|00111100) + -0.004+0.081j|01001011) + -0.077-0.085j|01001110) + 0.081+0.004j|01011010) + 0.004-0.081j|01101001) + -0.010+0.222j|01101010) + -0.077-0.085j|01101100) + 0.081+0.004j|01111000) + 0.081+0.004j|10000111) + -0.222-0.010j|10001011) + 0.222+0.010j|10001110) + 0.085-0.077j|10010011) + -0.004+0.081j|10010110) + -0.222-0.010j|10011010) + -0.010+0.222j|10100011) + -0.081-0.004j|10100101) + 0.010-0.222j|10100110) + 0.222+0.010j|10101001) + 0.222+0.010j|10101100) + -0.085+0.077j|10110001) + 0.010-0.222j|10110010) + -0.004+0.081j|10110100) + -0.222-0.010j|10111000) + 0.004-0.081j|11000011) + 0.077+0.085j|11000110) + -0.222-0.010j|11001010) + 0.081+0.004j|11010010) + 0.004-0.081j|11100001) + -0.010+0.222j|11100010) + -0.077-0.085j|11100100) + 0.222+0.010j|11101000) + -0.081

## Total excitation

In [14]:
n_basis = np.einsum('nij,njk->ik', dagger(phi_dense), phi_dense)
k_basis = np.einsum('nij,njk->ik', dagger(fock_dense)[:half_lat], fock_dense[:half_lat])
k_basis += np.einsum('nij,njk->ik', fock_dense[half_lat:], dagger(fock_dense)[half_lat:])

np.allclose(n_basis, k_basis)

True